In [2]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import ShuffleSplit
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the boston housing dataset
data = pd.read_csv('housing.csv')
prices = data['MEDV']
features = data.drop('MEDV', axis=1)

print "Boston housing dataset has {} data points with {} variables each.".format(*data.shape)
print "Axes are: ", data.axes

Boston housing dataset has 489 data points with 4 variables each.
Axes are:  [RangeIndex(start=0, stop=489, step=1), Index([u'RM', u'LSTAT', u'PTRATIO', u'MEDV'], dtype='object')]


## Data Exploration
### Calculate Statistics

In [3]:
# #Data Exploration
# ##Calculate Statisctics

minimum_price = prices.min()
maximum_price = prices.max()
mean_price = prices.mean()
median_price = prices.median()
std_price = prices.std()

print "Statistics for Boston housing datasets:\n"
print "Minimum price: ${:,.2f}".format(minimum_price)
print "Maximum price: ${:,.2f}".format(maximum_price)
print "Mean price: ${:,.2f}".format(mean_price)
print "Median price: ${:,.2f}".format(median_price)
print "Standard deviation of prices: ${:,.2f}".format(std_price)

Statistics for Boston housing datasets:

Minimum price: $105,000.00
Maximum price: $1,024,800.00
Mean price: $454,342.94
Median price: $438,900.00
Standard deviation of prices: $165,340.28


### Question 1 - Feature Observation
- `RM`: If the average number of rooms among homes in the neighborhood increases, then the value of the house should increase too. More number of rooms would lead to bigger houses. Bigger houses should be more expensive than small houses.
- `LSTAT`: If the percentage of "lower class" homeowners in the neighborhood increases, then the value of the house should decrease. If there are a lot of homeowners that cannot afford to expensive houses in the neighborhood, it would mean that the neighborhood has cheaps houses.
- `PTRATIO`: If the ratio of students to teachers increases, it would mean that there will be more and more students per teacher, which will cause a decrease in the value of the houses in the neighborhood. Higher class people will be willing to pay more for a more personalized education.
---

## Developing a Model
### Implementation: Define a Performance Metric

In [4]:
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """Calculates and returns the performance score between
       true and predicted values based on the metric chosen."""
    score = r2_score(y_true, y_predict)
    return score

### Question 2 - Foodness of Fit

In [5]:
# Calculate the performance of this model
score = performance_metric([3, -0.5, 2, 7, 4.2], [2.5, 0.0, 2.1, 7.8, 5.3])
print "Model has a coefficient of determination, R^2, of {:.3f}.".format(score)

Model has a coefficient of determination, R^2, of 0.923.


### Implementation: Shuffle and Split Data